# Text Summarization

# Import needed modules

In [1]:
#import the libraries
from nltk.corpus import stopwords 
from nltk.cluster.util import cosine_distance 
import numpy as np
import networkx as nx

In [2]:
!pip install spacy

In [3]:
import spacy

!python -m spacy download el_core_news_sm
!python -m spacy download de_core_news_sm
!python -m spacy download es_core_news_sm


nlp_greek = spacy.load("el_core_news_sm")
nlp_german = spacy.load("de_core_news_sm")
nlp_italian = spacy.load("es_core_news_sm")


#path for 3 files
textfiles = [
    r'C:\Users\DELL\Desktop\Greek.txt',
    r"C:\Users\DELL\Desktop\German.txt",
    r"C:\Users\DELL\Desktop\Italian.txt",
]

# Create lists  for each file
sentences_lists = [[] for _ in range(len(textfiles))]

# Function to tokenize sentences and identify language using spaCy
def tokenize_sentences_and_identify_language(text, nlp):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    language = doc.lang_
    return sentences, language

# Loop through each file and tokenize sentences
for index, filepath in enumerate(textfiles):
    with open(filepath, "r", encoding="utf-8") as file:
        filedata = file.readline().strip()  # Read only the first line (paragraph)
        sentences, language = tokenize_sentences_and_identify_language(filedata, 
                                                                       nlp_greek 
                                                                       if "Greek" in filepath 
                                                                       else (nlp_german 
                                                                       if "german" in filepath 
                                                                       else nlp_italian))
        sentences_lists[index].extend(sentences)
        print(f"Language identified for {filepath}: {language}")

# Print the sentences for each file
for index, sentences in enumerate(sentences_lists):
    print(f"\nSentences for file {textfiles[index]}:")
    for sentence in sentences:
        print(sentence)


     ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
     --------------------------------------- 0.0/12.6 MB 435.7 kB/s eta 0:00:29
     --------------------------------------- 0.1/12.6 MB 837.8 kB/s eta 0:00:15
     - -------------------------------------- 0.5/12.6 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.5/12.6 MB 6.6 MB/s eta 0:00:02
     ------ --------------------------------- 1.9/12.6 MB 7.7 MB/s eta 0:00:02
     ---------- ----------------------------- 3.2/12.6 MB 10.2 MB/s eta 0:00:01
     ------------- -------------------------- 4.2/12.6 MB 11.7 MB/s eta 0:00:01
     --------------- ------------------------ 5.0/12.6 MB 12.6 MB/s eta 0:00:01
     -------------------- ------------------- 6.4/12.6 MB 14.6 MB/s eta 0:00:01
     -------------------------- ------------- 8.2/12.6 MB 16.9 MB/s eta 0:00:01
     ----------------------------- ---------- 9.3/12.6 MB 1

In [5]:
sentences_lists

[['Τα τελευταία χρόνια, οι εξελίξεις στην τεχνητή νοημοσύνη (AI) έχουν φέρει επανάσταση σε διάφορους κλάδους, συμπεριλαμβανομένης της υγειονομικής περίθαλψης, των οικονομικών και των μεταφορών.',
  'Τεχνολογίες AI, όπως η μηχανική εκμάθηση και η επεξεργασία φυσικής γλώσσας, χρησιμοποιούνται για τον εξορθολογισμό των διαδικασιών, τη βελτίωση της λήψης αποφάσεων και Βελτιώστε τις εμπειρίες των πελατών, από την εξατομικευμένη ιατρική και την ανίχνευση απάτης έως τα αυτόνομα οχήματα και τους εικονικούς βοηθούς, η τεχνητή νοημοσύνη αναδιαμορφώνει τον τρόπο λειτουργίας των επιχειρήσεων και την αλληλεπίδραση των ατόμων με την τεχνολογία.',
  'και το μέλλον της εργασίας',
  '.'],
 ['Stell dir vor, es gibt keinen Himmel.',
  'Es ist einfach, wenn du es versuchst.',
  'Keine Hölle unter uns.',
  'Über uns nur Himmel.',
  'Stellen Sie sich all die Menschen vor, die für heute leben.',
  'Stellen Sie sich vor es gibt keine Länder.',
  'Das ist nicht schwer.',
  'Nichts, wofür man töten oder sterben

In [6]:
for i in range(len(sentences_lists)):
    print(f'-------------Text{i}--------------')
    print(' '*10)
    print(sentences_lists[i])
    print(' '*10)

-------------Text0--------------
          
['Τα τελευταία χρόνια, οι εξελίξεις στην τεχνητή νοημοσύνη (AI) έχουν φέρει επανάσταση σε διάφορους κλάδους, συμπεριλαμβανομένης της υγειονομικής περίθαλψης, των οικονομικών και των μεταφορών.', 'Τεχνολογίες AI, όπως η μηχανική εκμάθηση και η επεξεργασία φυσικής γλώσσας, χρησιμοποιούνται για τον εξορθολογισμό των διαδικασιών, τη βελτίωση της λήψης αποφάσεων και Βελτιώστε τις εμπειρίες των πελατών, από την εξατομικευμένη ιατρική και την ανίχνευση απάτης έως τα αυτόνομα οχήματα και τους εικονικούς βοηθούς, η τεχνητή νοημοσύνη αναδιαμορφώνει τον τρόπο λειτουργίας των επιχειρήσεων και την αλληλεπίδραση των ατόμων με την τεχνολογία.', 'και το μέλλον της εργασίας', '.']
          
-------------Text1--------------
          
['Stell dir vor, es gibt keinen Himmel.', 'Es ist einfach, wenn du es versuchst.', 'Keine Hölle unter uns.', 'Über uns nur Himmel.', 'Stellen Sie sich all die Menschen vor, die für heute leben.', 'Stellen Sie sich vor es gibt ke

In [7]:
# step-2 to find similarity between two sentences 
def sentence_similarity(sent1, sent2 ):
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        vector1[all_words.index(w)] += 1
        # build the vector for the second sentence
    for w in sent2:
        vector2[all_words.index(w)] += 1
    return 1 - cosine_distance(vector1, vector2)

In [8]:
similarity=[]
for i in sentences_lists:
    similarity_matrix = np.zeros((len(i), len(i)))
    for idx1 in range(len(i)):
        for idx2 in range(len(i)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(i[idx1], i[idx2])
    similarity.append(similarity_matrix)

In [9]:
similarity

[array([[0.        , 0.97588867, 0.8821835 , 0.02296576],
        [0.97588867, 0.        , 0.90312318, 0.00952122],
        [0.8821835 , 0.90312318, 0.        , 0.        ],
        [0.02296576, 0.00952122, 0.        , 0.        ]]),
 array([[0.        , 0.81558693, 0.8308194 , 0.80817083, 0.93567391,
         0.95536861, 0.71360064, 0.81687018, 0.81763179, 0.92681073,
         0.87049999, 0.88651573, 0.93515759, 0.75636501, 0.84435171,
         0.65950727, 0.9328115 ],
        [0.81558693, 0.        , 0.81609819, 0.75139136, 0.89479181,
         0.88878583, 0.87306626, 0.86299376, 0.76509206, 0.87699457,
         0.81003768, 0.94130404, 0.7778363 , 0.82152122, 0.74353583,
         0.75023448, 0.86958791],
        [0.8308194 , 0.81609819, 0.        , 0.82495791, 0.88883654,
         0.86713765, 0.59633024, 0.74560114, 0.86      , 0.87942585,
         0.75235985, 0.82697445, 0.79299257, 0.62853936, 0.81175997,
         0.78446454, 0.87782626],
        [0.80817083, 0.75139136, 0.82495791

In [10]:
# Step 3 - Rank sentences in similarity martix
score=[]
for i in similarity:
    sentence_similarity_graph = nx.from_numpy_array(i)
    scores = nx.pagerank(sentence_similarity_graph)
    score.append(scores)

In [11]:
score

[{0: 0.33196932675139845,
  1: 0.3241142672869578,
  2: 0.30158238808535853,
  3: 0.042334017876285254},
 {0: 0.06133371075937599,
  1: 0.060389010120109284,
  2: 0.05850821804244085,
  3: 0.05550288233760833,
  4: 0.06248454894001058,
  5: 0.062370618087784165,
  6: 0.05310405145484045,
  7: 0.057670245142359314,
  8: 0.058233200646216,
  9: 0.06194161152913476,
  10: 0.05972948279750336,
  11: 0.062206249081187846,
  12: 0.058705439934932956,
  13: 0.05462850704299813,
  14: 0.058080108930167,
  15: 0.053420502304462394,
  16: 0.06169161284886854},
 {0: 0.04565387857989269,
  1: 0.04565387857989269,
  2: 0.03921566322612661,
  3: 0.041314120265502345,
  4: 0.04111208010518796,
  5: 0.045085201313071796,
  6: 0.04250162640002954,
  7: 0.046381293806157894,
  8: 0.048816192988148904,
  9: 0.04053215888692057,
  10: 0.044902221207445074,
  11: 0.044619157698813736,
  12: 0.048816192988148904,
  13: 0.046381293806157894,
  14: 0.04549574069332334,
  15: 0.04680810446161505,
  16: 0.04698

In [12]:
ranked_sentences = []
for sentence_list in sentences_lists:
    ranked_sentence = sorted(((score, sentence) for score, sentence in zip(scores, sentence_list)), reverse=True)
    ranked_sentences.append(ranked_sentence)

In [13]:
ranked_sentences

[[(3, '.'),
  (2, 'και το μέλλον της εργασίας'),
  (1,
   'Τεχνολογίες AI, όπως η μηχανική εκμάθηση και η επεξεργασία φυσικής γλώσσας, χρησιμοποιούνται για τον εξορθολογισμό των διαδικασιών, τη βελτίωση της λήψης αποφάσεων και Βελτιώστε τις εμπειρίες των πελατών, από την εξατομικευμένη ιατρική και την ανίχνευση απάτης έως τα αυτόνομα οχήματα και τους εικονικούς βοηθούς, η τεχνητή νοημοσύνη αναδιαμορφώνει τον τρόπο λειτουργίας των επιχειρήσεων και την αλληλεπίδραση των ατόμων με την τεχνολογία.'),
  (0,
   'Τα τελευταία χρόνια, οι εξελίξεις στην τεχνητή νοημοσύνη (AI) έχουν φέρει επανάσταση σε διάφορους κλάδους, συμπεριλαμβανομένης της υγειονομικής περίθαλψης, των οικονομικών και των μεταφορών.')],
 [(16, 'Stellen Sie sich alle Menschen vor, die die ganze Welt teilen.'),
  (15, 'Eine Männerfreundschaft.'),
  (14, 'Keine Notwendigkeit für Gier oder Hunger.'),
  (13, 'Ich frage mich, ob du das kannst.'),
  (12, 'Stell dir keine Besitztümer vor.'),
  (11,
   'Ich hoffe, dass Sie sich uns e

In [14]:
# Finish off by printing summary.

for i in ranked_sentences:
    n = int(input("How many sentences do you want in the summary? "))
  
    for i in range(n):
        print("".join(ranked_sentence[i][1]))

How many sentences do you want in the summary? 2
NON MI PIACCIONO, SAM-I-AM.
NON MI PIACCIONO LE UOVA VERDI E IL PROSCIUTTO.
How many sentences do you want in the summary? 4
NON MI PIACCIONO, SAM-I-AM.
NON MI PIACCIONO LE UOVA VERDI E IL PROSCIUTTO.
NON MI PIACCIONO DA NESSUNA PARTE.
NON MI PIACCIONO QUI O LÀ.
How many sentences do you want in the summary? 6
NON MI PIACCIONO, SAM-I-AM.
NON MI PIACCIONO LE UOVA VERDI E IL PROSCIUTTO.
NON MI PIACCIONO DA NESSUNA PARTE.
NON MI PIACCIONO QUI O LÀ.
NON MI PIACCIONO CON IL MOUSE.
NON MI PIACCIONO IN CASA.
